## Import Statements

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Load DataFrame


In [ ]:
test = pd.read_csv('../input/hcd-cropped/sample_submission.csv')

print('Test Set Size:', test.shape)

In [ ]:
test.head()

## Extract Test Images

In [ ]:
test_path = "../input/hcd-cropped/test"
print('Test Images:', len(os.listdir(test_path)))

In [ ]:
def append_ext(fn):
    return fn+".tif"

In [ ]:
test['id'] = test['id'].apply(append_ext)
test.head()

## Data Generator

In [ ]:
BATCH_SIZE = 128

test_datagen = ImageDataGenerator(rescale=1/255)

test_loader = test_datagen.flow_from_dataframe(
    dataframe = test,
    directory = test_path,
    x_col = 'id',
    batch_size = BATCH_SIZE,
    shuffle = False,
    class_mode = None,
    target_size = (32,32)
)

## Load Model

In [ ]:
cnn = keras.models.load_model('../input/cancer-detection-crop-models/cancer_detection_model_crop_v07.h5')
                               
cnn.summary()

## Test Predictions

In [ ]:
test_probs = cnn.predict(test_loader)
print(test_probs.shape)

In [ ]:
print(test_probs[:10,].round(2))

## Prepare Submission

In [ ]:
submission = pd.read_csv('../input/hcd-cropped/sample_submission.csv')
submission.head()

In [ ]:
submission.label = test_probs[:,1]
submission.head()

In [ ]:
submission.to_csv('submission.csv', header=True, index=False)